In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# loading the model

In [ ]:
ds = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin1')

In [ ]:
ds.head()

In [ ]:
ds.info()

# feature selection

In [ ]:
ds.isnull().sum()

In [ ]:
ds = ds.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [ ]:
ds.info()

In [ ]:
ds.columns = ['cls','email']

In [ ]:
ds.head()

## cheking duplicates

In [ ]:
ds.duplicated().sum()

In [ ]:
ds = ds.drop_duplicates(keep='first')

In [ ]:
ds.duplicated().sum()

## encoding categorical values

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
ds['cls'] = le.fit_transform(ds['cls'])

In [ ]:
ds.cls.value_counts()

##  ham : 0 ; spam : 1

In [ ]:
ds.head()

In [ ]:
ham = ds[ds['cls']==0]
spam = ds[ds['cls']==1]
ham.head(),spam.head()

In [ ]:
ham_txt = " ".join(ham.email)
ham_txt[0:25]

In [ ]:
spam_txt = " ".join(spam.email)
spam_txt[0:25]

# visualization

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,7))
wc = WordCloud(width=800,height=650).generate(ham_txt)
plt.imshow(wc)
plt.axis('off')
plt.title('word cloud of ham emails')
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
wc = WordCloud(width=800,height=650).generate(spam_txt)
plt.imshow(wc)
plt.axis('off')
plt.title('word cloud of spam emails')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x = ds['cls'])
plt.xlabel('labels')
plt.ylabel('count')
plt.title('count of classes of emails')
plt.show()

# preprocessing pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
model = Pipeline([
    ('vectorizer',TfidfVectorizer(
    tokenizer = lambda txt:[token.lemma_ for token in nlp(txt)],
    stop_words = 'english',
    ngram_range=(1,2),
    max_features=500))
])

In [ ]:
x_prep = model.fit_transform(ds['email'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_prep,ds['cls'],test_size=0.2,random_state=42,stratify=ds['cls'])

In [ ]:
x_train.shape,x_test.shape

In [ ]:
max_words = x_prep.shape[1]  # Number of words in the TF-IDF vocabulary
max_length = x_prep.shape[1]  # Maximum sequence length

# model creation

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense,Embedding,Masking

In [ ]:
dl_model = Sequential([
    Embedding(input_dim=max_words,output_dim=128,input_length=max_length,mask_zero=True),
    Masking(mask_value=0.0),
    LSTM(64,return_sequences=True),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
dl_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
x_train = x_train.toarray()
x_test = x_test.toarray()

In [ ]:
epochs = 5
batch_size = 64
history = dl_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

# learning curve

In [ ]:
def learning_curve(history, epoch):
    # training vs validation accuracy
    epoch_range = range(1,epoch+1)
    plt.plot(epoch_range, history.history['accuracy'])
    plt.plot(epoch_range, history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()

# training vs validation loss
    plt.plot(epoch_range, history.history['loss'])
    plt.plot(epoch_range, history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'val'], loc='upper left')
    plt.show()

In [ ]:
learning_curve(history, 5)